<a href="https://colab.research.google.com/github/Deeksha-Pandit/EVA4-S12/blob/master/Assignment%20A/S12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<h3>Install Albumentations and Apex libraries</h3>**

In [1]:
!pip install -U git+https://github.com/albu/albumentations
#!pip install --no-cache-dir git+https://github.com/NVIDIA/apex

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-t1yjpcwi
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-t1yjpcwi
     |████████████████████████████████| 634kB 6.4MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=65099 sha256=fc2299951c9e7f40b5871dc4760df1009b087f19794c7be15c84a9ad798d22b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-8urlp0jg/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=dc8faa4b59d472de7dfa78ad101c2ccbda4075a3f23a135dbc068374ea06c17f
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
  

**<h3>Mount Drive and unzip data<h3>**

In [2]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
folder = '/content/gdrive/My Drive/Session12'
sys.path.append(folder)

**<h3>Import required files<h3>**

In [0]:
import Albumentationtransform
import my_model
import train_test
import LR_Range_Test
import train_test_loader
import tinyimagenet
import evaluate
import GradCam
import show_images

In [5]:
import albumentations as A
import cv2
channel_means = (0.5, 0.5, 0.5)
channel_stdevs = (0.5, 0.5, 0.5)
train_transform = Albumentationtransform.AlbumentationTransforms([       
                                                                
                                      A.HorizontalFlip(p = 0.7),
                                      A.PadIfNeeded(min_height=70, min_width=70, border_mode=4, value=None, mask_value=None, always_apply=False, p=1.0),
                                      #A.GaussNoise(var_limit=(10.0, 50.0), mean=0, always_apply=False, p=0.5),
                                      A.RandomCrop(64, 64, always_apply=False, p=1.0),
                                      # A.transforms.ShiftScaleRotate()
                                      A.Normalize(mean=channel_means, std=channel_stdevs),
                                      A.Cutout(num_holes=1, max_h_size=32,max_w_size = 32,p=0.7) 
                                       ])
test_transform = Albumentationtransform.AlbumentationTransforms([A.Normalize(mean=channel_means, std=channel_stdevs)])
train_dataset , test_dataset,classes = tinyimagenet.TinyImageNetDataSet(train_split = 70,test_transforms = test_transform,train_transforms = train_transform)

In [0]:
trainloader, testloader = train_test_loader.load(train_dataset,test_dataset,batch_size=256)

In [7]:
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = my_model.ResNet18(num_classes=200).to(device)
summary(model, input_size=(3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
            Conv2d-3           [-1, 64, 64, 64]          36,864
       BatchNorm2d-4           [-1, 64, 64, 64]             128
            Conv2d-5           [-1, 64, 64, 64]          36,864
       BatchNorm2d-6           [-1, 64, 64, 64]             128
        BasicBlock-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
           Conv2d-10           [-1, 64, 64, 64]          36,864
      BatchNorm2d-11           [-1, 64, 64, 64]             128
       BasicBlock-12           [-1, 64, 64, 64]               0
           Conv2d-13          [-1, 128, 32, 32]          73,728
      BatchNorm2d-14          [-1, 128,

In [0]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR

optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9,weight_decay=0.005 ) 
criterion = nn.CrossEntropyLoss()
scheduler = OneCycleLR(optimizer, max_lr = 0.02, total_steps=None, epochs=50, steps_per_epoch=1, pct_start=0.3, anneal_strategy='linear', cycle_momentum=False, base_momentum=0.85, max_momentum=0.95, div_factor=10.0,final_div_factor =1)

In [9]:
train_test.training(model,device,trainloader,testloader,optimizer,criterion,50,scheduler)

EPOCH: 1 LR: 0.002



Test set: Average loss: 0.0179, Accuracy: 2292/33000 (6.95%)

EPOCH: 2 LR: 0.003285714285714286



Test set: Average loss: 0.0161, Accuracy: 3827/33000 (11.60%)

EPOCH: 3 LR: 0.004571428571428572



Test set: Average loss: 0.0155, Accuracy: 4583/33000 (13.89%)

EPOCH: 4 LR: 0.005857142857142858



Test set: Average loss: 0.0155, Accuracy: 4822/33000 (14.61%)

EPOCH: 5 LR: 0.0071428571428571435


...


Test set: Average loss: 0.0079, Accuracy: 16813/33000 (50.95%)

EPOCH: 47 LR: 0.0035428571428571427



Test set: Average loss: 0.0076, Accuracy: 17403/33000 (52.74%)

EPOCH: 48 LR: 0.0030285714285714256



Test set: Average loss: 0.0075, Accuracy: 17727/33000 (53.72%)

EPOCH: 49 LR: 0.002514285714285712



Test set: Average loss: 0.0071, Accuracy: 18260/33000 (55.33%)

EPOCH: 50 LR: 0.0019999999999999983



Test set: Average loss: 0.0070, Accuracy: 18707/33000 (56.69%)

